In [1]:
from DiSuQ.Torch import models
from torch import tensor,stack
from numpy import arange,linspace,meshgrid,array,log,argsort,pi
from DiSuQ.utils import plotCompare,plotHeatmap,plotBox
from DiSuQ.Torch.optimization import uniformParameters,truncNormalParameters,initializationSequential
from DiSuQ.Torch import optimization
from DiSuQ.Torch.optimization import lossDegeneracyTarget,lossDegeneracyWeighted
from DiSuQ.Torch.components import indE,capE
from torch import set_num_threads
set_num_threads(32)

In [2]:
pairs = {'Lx':'Ly','Cx':'Cy','Jx':'Jy','CJx':'CJy'}

In [3]:
L_ = indE(1e-7); L0 = indE(5*1e-9); print('Inductance Bound(GHz):',L_,L0+L_)
C0 = capE(5*1e-16); C_ = capE(1e-14*5) ; print('Capacitance Bound(GHz):',C_,C0+C_)
CJ0 = capE(5*1e-16/4); CJ_ = capE(1e-14/2) ; print('Shunt Bound(GHz):',CJ_,CJ0+CJ_)
J0 = 25. ; J_ = 5. ; print('Junction Bound(GHz):',J_,J0+J_)
# components['Jx'].J0 = J0 ; components['Jy'].J0 = J0

Inductance Bound(GHz): 1.6346151294983178 34.32691771946467
Capacitance Bound(GHz): 0.38740458615415185 39.12786320156934
Shunt Bound(GHz): 3.8740458615415188 158.83588032320228
Junction Bound(GHz): 5.0 30.0


In [4]:
basis = [7,7,7] ; rep = 'Q'
flux_point = ['Lx','Ly']
circuit = models.zeroPi(basis,Ej=10.,Ec=5.,El=10.,EcJ=50.,sparse=False,symmetry=True,
                        _L_=(L_,L0),_C_=(C_,C0),_J_=(J_,J0),_CJ_=(CJ_,CJ0))
static = circuit.circuitState()

In [5]:
flux_profile = [{'Lx':tensor(.5),'Ly':tensor(0.0)},{'Lx':tensor(.45),'Ly':tensor(0.0)}]

In [6]:
def optimizationAnalysis(init,subspace,Search,success=1.):
    Loss,Success,Paths = [],[],[]
    for index,(init,(dLogs,dParams,dCircuit)) in enumerate(zip(init,Search)):
        Paths.append(dCircuit[subspace].to_numpy())
        loss = dLogs['loss'].to_numpy()
        Loss.append(loss[-1])
        if loss[-1] < success:
            Success.append(len(loss))
    return Paths,Loss,Success

In [7]:
def analysisPlotting(Optimization,Algo=['LBFGS']):
    paths = dict()
    losse = dict()
    for algo,(Paths,Loss,Success) in zip(Algo,Optimization):
        indices = argsort(Loss)[0]
        paths[algo] = Paths[indices]
        losse[algo] = Loss
    return paths,losse

In [8]:
def lossScapeBounds(paths):
    Paths = []
    for algo,path in paths.items():
        Paths.append(path)
    Paths = vstack(Paths)
    Paths = Paths[:, ~isnan(Paths).any(axis=0)]
    return Paths.min(0),Paths.max(0)

In [9]:
N = 10; subspace = ['Lx','Cx','Jx','CJx']
initials = truncNormalParameters(circuit,subspace,N=N,var=8)
len(initials)

10

In [10]:
N = 3; subspace = ['Lx','Cx','Jx','CJx']
initials = uniformParameters(circuit,subspace,N)
len(initials)

81

In [11]:
optimizer = optimization.OrderingOptimization(circuit,representation=rep)

### Degeneracy domination

In [12]:
D0 = 1; delta0 = 0
lossFunction = lossDegeneracyWeighted(delta0,D0)

In [ ]:
LBFGS_D = []
for index,parameter in enumerate(initials):
    print(optimizer.circuitState())
    print(index,parameter)
    optimizer.circuit.initialization(parameter)  
    optimizer.parameters,optimizer.IDs = optimizer.circuitParameters()
    LBFGS_D.append(optimizer.minimization(lossFunction,flux_profile,
                    method='L-BFGS-B',options=dict(ftol=1e-16,maxiter=15)))

{'Lx': 10.0, 'Ly': 10.0, 'Cx': 5.0, 'Cy': 5.0, 'Jx': 10.0, 'Jy': 10.0, 'CJx': 50.0, 'CJy': 50.0}
0 {'Lx': 8.173075647491588, 'Ly': 10.0, 'Cx': 9.685114653853796, 'Cy': 5.0, 'Jx': 6.25, 'Jy': 10.0, 'CJx': 38.740458615415186, 'CJy': 50.0}
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            4     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f= -1.38786D-01    |proj g|=  1.11256D-01


In [ ]:
Result = [optimizationAnalysis(initials,subspace,LBFGS_D)]
paths_D,losse_D = analysisPlotting(Result)

In [24]:
plotBox(losse_D,'Loss-Degeneracy Distribution',export='pdf',size=(300,400))

### Delta domination

In [25]:
D0 = 0; delta0 = 1
lossFunction = lossDegeneracyWeighted(delta0,D0)

In [ ]:
LBFGS_Delta = []
for index,parameter in enumerate(initials):
    print(optimizer.circuitState())
    print(index,parameter)
    optimizer.circuit.initialization(parameter)  
    optimizer.parameters,optimizer.IDs = optimizer.circuitParameters()
    LBFGS_Delta.append(optimizer.minimization(lossFunction,flux_profile,
                    method='L-BFGS-B',options=dict(ftol=1e-16,maxiter=15)))

{'Lx': 16.345783233642578, 'Ly': 16.345783233642578, 'Cx': 19.370576858520508, 'Cy': 19.370576858520508, 'Jx': 6.2498321533203125, 'Jy': 6.2498321533203125, 'CJx': 116.22157287597656, 'CJy': 116.22157287597656}
0 {'Lx': 8.173075647491588, 'Ly': 10.0, 'Cx': 9.685114653853796, 'Cy': 5.0, 'Jx': 6.25, 'Jy': 10.0, 'CJx': 38.740458615415186, 'CJy': 50.0}
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            4     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f= -6.58795D+00    |proj g|=  8.93699D-01

At iterate    1    f= -6.84227D+00    |proj g|=  8.23959D-01


/Users/chishti/JupyterHub/DiSuQ/Torch/components.py:72: RuntimeWarning:

divide by zero encountered in scalar divide




At iterate    2    f= -1.00987D+01    |proj g|=  1.23035D+00

At iterate    3    f= -1.02568D+01    |proj g|=  7.68045D-01

At iterate    4    f= -1.13030D+01    |proj g|=  2.32183D+00
  ys=-2.280E+00  -gs= 1.282E+00 BFGS update SKIPPED

At iterate    5    f= -1.60391D+01    |proj g|=  0.00000D+00

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    4      5     13      9     1     4   0.000D+00  -1.604D+01
  F =  -16.039117813110352     

CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL            
{'Lx': 1.634615182876587, 'Ly': 1.634615182876587, 'Cx': 39.127864837646484, 'Cy': 39.127864837646484, 'Jx': 5.0, 'Jy': 5.0


   evaluations in the last line search.  Termination
   may possibly be caused by a bad search direction.


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            4     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f= -8.16274D+00    |proj g|=  1.19679D+00

At iterate    1    f= -8.92018D+00    |proj g|=  1.94632D+00
  ys=-1.116E+00  -gs= 2.715E+00 BFGS update SKIPPED

At iterate    2    f= -8.92018D+00    |proj g|=  1.94632D+00


...

RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            4     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f= -7.05238D+00    |proj g|=  5.41940D-01

At iterate    1    f= -7.07219D+00    |proj g|=  5.05530D-01

At iterate    2    f= -9.35590D+00    |proj g|=  8.78166D-01
  ys=-6.721E-01  -gs= 8.675E+00 BFGS update SKIPPED

At iterate    3    f= -1.05950D+01    |proj g|=  8.41957D-02

At iterate    4    f= -1.14038D+01    |proj g|=  3.74266D-01
  ys=-4.030E-01  -gs= 1.170E-01 BFGS update SKIPPED

At iterate    5    f= -1.23249D+01    |proj g|=  1.00567D+00
  ys=-3.927E+00  -gs= 2.312E+00 BFGS update SKIPPED



 Bad direction in the line search;
   refresh the lbfgs memory and restart the iteration.



At iterate    6    f= -1.23249D+01    |proj g|=  1.00567D+00

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
{'Lx': 1.7618746757507324, 'Ly': 1.7618746757507324, 'Cx': 39.127864837646484, 'Cy': 39.127864837646484, 'Jx': 5.0, 'Jy': 5.0, 'CJx': 122.19636535644531, 'CJy': 122.19636535644531}
78 {'Lx': 24.519226942474766, 'Ly': 10.0, 'Cx': 29.05534396156139, 'Cy': 5.0, 'Jx': 18.75, 'Jy': 10.0, 'CJx': 38.740458615415186, 'CJy': 50.0}
    4      6     55      9     3     1   1.006D+00  -1.232D+01
  F =  -12.324901580810547     

CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH             



   evaluations in the last line search.  Termination
   may possibly be caused by a bad search direction.


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            4     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f= -5.25375D+00    |proj g|=  8.41127D-01

At iterate    1    f= -5.34429D+00    |proj g|=  8.21215D-01

At iterate    2    f= -9.56485D+00    |proj g|=  1.04150D+00
  ys=-6.642E+00  -gs= 4.234E+01 BFGS update SKIPPED

At iterate    3    f= -9.65255D+00    |proj g|=  1.23035D-01

At iterate    4    f= -1.59806D+01    |proj g|=  8.78008D+01
  ys=-2.912E+02  -gs= 3.163E-01 BFGS update SKIPPED


In [ ]:
Result = [optimizationAnalysis(initials,subspace,LBFGS_Delta)]
paths_delta,losse_delta = analysisPlotting(Result)

In [ ]:
plotBox(losse_delta,'Loss-Delta Distribution',export='pdf',size=(300,400))

### Comparison

In [ ]:
paths['optimal']  = [array([1.1*L_,1.1*C_,J0+.9*J_,CJ0+.9*CJ_])]

In [ ]:
paths.update(paths_delta)
paths.update(paths_D)

In [ ]:
Degeneracy,plots = dict(),dict()
for algo,path in paths.items():
    parameters = dict(zip(subspace,path[-1]))
    static.update(parameters)
    circuit.initialization(static)
    H_LC = circuit.chargeHamiltonianLC()
    H_J = circuit.josephsonCharge
    E0,(Ex1,Ex2) = circuit.spectrumManifold(flux_point,flux_manifold,H_LC,H_J,excitation=[1,2],grad=True)
    Ex2 = Ex2.detach().numpy()
    Ex1 = Ex1.detach().numpy()
    degeneracy = log(Ex2/Ex1)
    Degeneracy[algo] = degeneracy
    plots[algo+'-E10'] = Ex1
    plots[algo+'-E20'] = Ex2

In [ ]:
plotCompare(flux_range[:,0],Degeneracy,'Degeneracy-Optimal Circuits','flux profile',export='pdf',size=(600,800))

In [ ]:
plotCompare(flux_range[:,0],plots,'Spectrum-Optimal Circuits','flux profile','energy(GHz)',export='pdf',size=(600,800))

## Multiple Solution